In [32]:
import pandas as pd
import statsmodels.api as sm

from scipy.stats import norm
from statsmodels.sandbox.regression.gmm import GMM


import numpy as np

In [34]:
data = pd.read_excel('ccapm.xlsx', header=None)
data.columns = ['cratio', 'rrate', 'e']

data['cratio1'] = data['cratio'].shift(1)
data['rrate1'] = data['rrate'].shift(1)
data['e1'] = data['e'].shift(1)
data = data.dropna()
data['constant'] = 1


In [74]:
endo_var = data[['cratio', 'rrate']].values 
inst_var = data[['constant', 'cratio1', 'rrate1', 'e1']].values 


### Problem (a)

In [80]:
class CCAPM(GMM):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.param_names = ["b", "r"]  
        
    def momcond(self, params):
        b, r = params
        cratio = self.endog[:, 0]
        rrate = self.endog[:, 1]
        constant = self.instrument[:, 0]
        cratio1 = self.instrument[:, 1]
        rrate1 = self.instrument[:, 2]
        e1 = self.instrument[:, 3]
        endo_equ = b * (cratio ** -r) * rrate - 1
        moment  = np.column_stack((
            endo_equ * constant,
            endo_equ * cratio1,
            endo_equ * rrate1,
            endo_equ * e1
        ))
        return moment
        
start_params = np.array([1, 1])  


CCAPM_model = CCAPM(endog = endo_var, exog = endo_var, instrument = inst_var, k_moms=4, k_params=2)
res = CCAPM_model.fit(start_params)


print(res.summary())


Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 1
         Function evaluations: 3
         Gradient evaluations: 3
Optimization terminated successfully.
         Current function value: 0.005432
         Iterations: 5
         Function evaluations: 10
         Gradient evaluations: 10
Optimization terminated successfully.
         Current function value: 0.005188
         Iterations: 4
         Function evaluations: 9
         Gradient evaluations: 9
Optimization terminated successfully.
         Current function value: 0.005197
         Iterations: 1
         Function evaluations: 3
         Gradient evaluations: 3
Optimization terminated successfully.
         Current function value: 0.005196
         Iterations: 1
         Function evaluations: 3
         Gradient evaluations: 3
                                CCAPM Results                                 
Dep. Variable:           ['y1', 'y2']   Hansen J:                        1

### Problem (b)

In [72]:
res_2 = CCAPM_model.fit(start_params,weights_method='hac', wargs={'maxlag':5})
print(res_2.summary())

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 1
         Function evaluations: 3
         Gradient evaluations: 3
Optimization terminated successfully.
         Current function value: 0.008025
         Iterations: 5
         Function evaluations: 10
         Gradient evaluations: 10
Optimization terminated successfully.
         Current function value: 0.007271
         Iterations: 4
         Function evaluations: 9
         Gradient evaluations: 9
Optimization terminated successfully.
         Current function value: 0.007278
         Iterations: 1
         Function evaluations: 3
         Gradient evaluations: 3
Optimization terminated successfully.
         Current function value: 0.007277
         Iterations: 1
         Function evaluations: 3
         Gradient evaluations: 3
                                CCAPM Results                                 
Dep. Variable:           ['y1', 'y2']   Hansen J:                        1